<a href="https://colab.research.google.com/github/ShakhovaP/musical-chord-recognition/blob/main/cnn_chord_recognition_signal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow as tf
import tensorflow.keras as keras
import matplotlib.pyplot as plt


DATASET_PATH = '/content/gdrive/MyDrive/dataset-2.json'




In [ ]:
labels = [
            "A",
            "A#m|Bbm",
            "A#|Bb",
            "Am",
            "B",
            "Bm",
            "C",
            "C#m|Dbm",
            "C#|Db",
            "Cm",
            "D",
            "D#m|Ebm",
            "D#|Eb",
            "Dm",
            "E",
            "Em",
            "F",
            "F#m|Gbm",
            "F#|Gb",
            "Fm",
            "G",
            "G#m|Abm",
            "G#|Ab",
            "Gm",
            "N",
                    ]

# def vectorize(number):
#     vector = [0 for _ in range(25)]
#     vector[number] = 1
#     return vector

def load_data(data_path):
    """Loads training dataset from json file.

        :param data_path (str): Path to json file containing data
        
        :return X (ndarray): Inputs
        :return y (ndarray): Targets
    """

    with open(data_path, "r") as fp:
        data = json.load(fp)

    # X = np.array(data["MFCCs"])
    X = np.array(data["signal"])
    y = np.array(data["chords"])
    return X, y


def plot_history(history):
    """Plots accuracy/loss for train and validation set as a function of the epochs
        :param history: Training history of model
        :return:
    """

    fig, axs = plt.subplots(2)

    # create accuracy sublpot
    axs[0].plot(history.history["accuracy"], label="train accuracy")
    axs[0].plot(history.history["val_accuracy"], label="test accuracy")
    axs[0].set_ylabel("Accuracy")
    axs[0].set_title("Accuracy eval")

    # create error sublpot
    axs[1].plot(history.history["loss"], label="train error")
    axs[1].plot(history.history["val_loss"], label="test error")
    axs[1].set_ylabel("Error")
    axs[1].set_xlabel("Epoch")
    axs[1].set_title("Error eval")

    plt.show()


def prepare_datasets(test_size, validation_size):
    """Loads data and splits it into train, validation and test sets.

    :param test_size (float): Value in [0, 1] indicating percentage of data set to allocate to test split
    :param validation_size (float): Value in [0, 1] indicating percentage of train set to allocate to validation split

    :return X_train (ndarray): Input training set
    :return X_validation (ndarray): Input validation set
    :return X_test (ndarray): Input test set
    :return y_train (ndarray): Target training set
    :return y_validation (ndarray): Target validation set
    :return y_test (ndarray): Target test set
    """

    # load data
    X, y = load_data(DATASET_PATH)

    # create train, validation and test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    # add an axis to input sets
    X_train = X_train[..., np.newaxis]
    X_validation = X_validation[..., np.newaxis]
    X_test = X_test[..., np.newaxis]

    return X_train, X_validation, X_test, y_train, y_validation, y_test


def build_model(input_shape):
    """Generates CNN model

    :param input_shape (tuple): Shape of input set
    :return model: CNN model
    """

    model = keras.Sequential()

    model.add(keras.layers.Conv1D(32, 3, activation='relu', input_shape=input_shape, padding='same'))
    model.add(keras.layers.MaxPooling2D(4, padding='same'))
    model.add(keras.layers.BatchNormalization())

    model.add(keras.layers.Conv1D(32, 3, activation='relu', input_shape=input_shape, padding='same'))
    model.add(keras.layers.MaxPooling2D(4, padding='same'))
    model.add(keras.layers.BatchNormalization())

    model.add(keras.layers.Conv1D(32, 3, activation='relu', input_shape=input_shape, padding='same'))
    model.add(keras.layers.MaxPooling2D(4, padding='same'))
    model.add(keras.layers.BatchNormalization())

    model.add(keras.layers.Flatten())

    model.add(keras.layers.Dense(25, activation='softmax'))

    

    model.add(keras.layers.Dense(25, activation='softmax'))

    return model


def predict(model, X, y):
    """Predict a single sample using the trained model

    :param model: Trained classifier
    :param X: Input data
    :param y (int): Target
    """

    # add a dimension to input data for sample - model.predict() expects a 4d array in this case
    X = X[np.newaxis, ...] 

    # perform prediction
    prediction = model.predict(X)

    # get index with max value
    predicted_index = np.argmax(prediction, axis=1)
    print("Probabilities:\n", prediction)
    index_t = int(np.where(y==1)[0])
    index_p = int(predicted_index[0])
    print(f"\nTarget: {labels[index_t]}, Predicted label: {labels[index_p]}")
    
   
    pred = np.array(prediction)
    sorted_p = pred.sort()
    
    


if __name__ == "__main__":

    # get train, validation, test splits
    X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.3, 0.2)
    
    mean = np.mean(X_train, axis=0)
    std = np.std(X_train, axis=0)
    X_train = (X_train - mean)/std
    
    mean = np.mean(X_validation, axis=0)
    std = np.std(X_validation, axis=0)    
    X_validation = (X_validation - mean)/std
    
    mean = np.mean(X_test, axis=0)
    std = np.std(X_test, axis=0)
    X_test = (X_test - mean)/std
    
    
    n_classes = 25
    y_train = keras.utils.to_categorical(y_train, n_classes)
    print(y_train)
    y_validation = keras.utils.to_categorical(y_validation, n_classes)
    y_test = keras.utils.to_categorical(y_test, n_classes)
    
    # create network
    input_shape = (X_train.shape[1], X_train.shape[2], 1)
    print(input_shape)
    model = build_model(input_shape)

    # compile model
    model.compile(optimizer="adam",
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    model.summary()

    # train model
    history = model.fit(X_train, 
                        y_train, 
                        validation_data=(X_validation, y_validation), 
                        batch_size=64, 
                        epochs=10)

    # plot accuracy/error for training and validation
    plot_history(history)

    # evaluate model on test set
    test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
    print('\nTest accuracy:', test_acc)

    # pick a sample to predict from the test set
    X_to_predict = X_test[100]
    y_to_predict = y_test[100]

    # predict sample
    predict(model, X_to_predict, y_to_predict)